<a href="https://colab.research.google.com/github/chengeke/AdaLovelaceCorner/blob/master/LoanStats_2018Q4(C).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded=files.upload()
for fn in uploaded.keys():
  print('user uploaded file"LoanStats_2018Q4(C).csv"with length 1,060,929bytes'.format(name=fn,length=len(uploaded[fn])))
 
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import SGD
import keras.utils
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import datetime
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder
np.random.seed(3)
 
df=pd.read_csv('LoanStats_2018Q4(C).csv')
print(df.head())
print(df.shape)
print(df.isnull().sum())
labelencoder=LabelEncoder()
df['loan_status']=labelencoder.fit_transform(df['loan_status'])

for column in df.columns:
    df['loan_status'] = labelencoder.fit_transform(df['loan_status'])
x = df.drop(columns=['loan_status'], axis=1)
y = df['loan_status']
random_state = np.random.RandomState(0)
n_samples, n_features = x.shape
x = np.c_[x, random_state.randn(n_samples, 200 * n_features)]
x, y = make_classification(n_samples=3000, n_classes=2, n_features=10, random_state=0)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.2, random_state=5)

forest = RandomForestClassifier()
forest.fit(x_train, y_train)
y_pred = forest.predict(x_test)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))
print("confusion matrixt:\n",confusion_matrix(y_test, y_pred))

rf = RandomForestClassifier(max_features=5, n_estimators=500)
rf.fit(x_train, y_train)
nb = GaussianNB()
nb.fit(x_train, y_train)
r_probs = [0 for _ in range(len(y_test))]
rf_probs = rf.predict_proba(x_test)
nb_probs = nb.predict_proba(x_test)
rf_probs = rf_probs[:, 1]
nb_probs = nb_probs[:, 1]
r_auc = roc_auc_score(y_test, r_probs)
rf_auc = roc_auc_score(y_test, rf_probs)
nb_auc = roc_auc_score(y_test, nb_probs)
r_fpr, r_tpr, _ = roc_curve(y_test, r_probs)
rf_fpr, rf_tpr, _ = roc_curve(y_test, rf_probs)
nb_fpr, nb_tpr, _ = roc_curve(y_test, nb_probs)

plt.plot(r_fpr, r_tpr, linestyle='--', label='Random prediction (AUROC = %0.3f)' % r_auc)
plt.plot(rf_fpr, rf_tpr, marker='.', label='Random Forest (AUROC = %0.3f)' % rf_auc)
plt.plot(nb_fpr, nb_tpr, marker='.', label='Naive Bayes (AUROC = %0.3f)' % nb_auc)
plt.title('ROC Plot for LoanStats_2018Q4(C)')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend() 
plt.show()
 
%reload_ext tensorboard
%tensorboard --logdir logs/fit
columns=['term','int_rate','grade','sub_grade','emp_title','emp_length','home_ownership',
         'verification_status','issue_d','loan_status','pymnt_plan','purpose','title',
         'zip_code','addr_state','earliest_cr_line','revol_util','initial_list_status',
         'last_pymnt_d','next_pymnt_d','last_credit_pull_d','application_type',
         'verification_status_joint','sec_app_earliest_cr_line','hardship_flag','debt_settlement_flag']
for var in columns:
    le = preprocessing.LabelEncoder()
    df[var]=le.fit_transform(df[var].astype('str'))
labelencoder=LabelEncoder()

x = df.drop(columns=['loan_status'], axis=1)
y = df['loan_status']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.25, random_state=45)

model = Sequential()
model.add(Dense(100, input_dim=110,  activation='relu'))
model.add(Dense(1, input_dim=110,  activation='relu'))

sgd=SGD(learning_rate=0.0001,decay=1e-3,momentum=0.75,nesterov=True)
model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['binary_accuracy'])


tbCallBack = keras.callbacks.TensorBoard(log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S"), histogram_freq=0, write_graph=True, write_images=False)

results= model.fit(x_train, y_train, validation_data=(x_test, y_test),
                   callbacks=[tbCallBack],epochs=20, batch_size=400)
epochs=20
epochs_arr = np.arange(1,epochs + 1, 1)
data_results =results.history
line1 = plt.plot(epochs_arr, data_results['loss'], 'r-', label='training loss')
line2 = plt.plot(epochs_arr, data_results['val_loss'], 'b-', label='validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Model loss')
plt.legend()
plt.show()

line1 = plt.plot(epochs_arr,data_results['binary_accuracy'], 'r-', label='training accuracy')
line2 = plt.plot(epochs_arr,data_results['val_binary_accuracy'], 'b-', label='validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Model accuracy')
plt.legend()
plt.show()

accuracy=model.evaluate(x_test,y_test)
print("accuracy=",(model.metrics_names[1],accuracy[1]*100))